In [ ]:
import os
import glob
import re
import shutil
import matplotlib.pyplot as plt
#import pydicom
import numpy as np
import pandas as pd
from PIL import Image,ImageFilter
import tempfile
import zipfile
import multiprocessing as mp
import json
import pydicom
import copy
from distutils.dir_util import copy_tree

In [ ]:
samples[0]

In [ ]:
cases=os.listdir("/mnt/idms/kdomokos/data/sinogram/20210901")

In [ ]:
def do_copy(case):
    copy_tree(f"/mnt/idms/kdomokos/data/sinogram/20210901/{case}", f"/mnt/idms/PROJECTS/Lung/sinogram_Domi/{case}")

In [ ]:
with mp.Pool(40) as pool:
    _ = pool.map(do_copy, cases)
pool.join()

In [ ]:
len(glob.glob("/mnt/idms/home/andor/prep_result/*"))

In [ ]:
case_path="/mnt/idms/PROJECTS/Lung/LIDC-IDRI/LIDC-IDRI-0009"
for slice_path in glob.iglob(f'{case_path}/*'):
    print(slice_path)
    dc=pydicom.dcmread(slice_path)

In [ ]:
type((f.pixel_array)[0][0])

In [ ]:
print(ff)

In [ ]:
slice_thickness

In [ ]:
slice_thickness=f.get_item((18, 50))

In [ ]:
l=[np.array([1,2]), np.array([3,4,5])]
for x in l:
    x[0]=100

In [ ]:
l

In [ ]:
l=glob.glob(f"/mnt/idms/PROJECTS/Lung/LIDC-IDRI-SuperResoluted/LIDC-IDRI-0151/*")
l=[pydicom.dcmread(p) for p in l]
l=sorted(l, key=lambda s: s.SliceLocation)
for d in l:
    arr=d.pixel_array
    plt.figure()
    plt.imshow(arr, cmap='gray')
    plt.show()

In [ ]:
np.where(np.array([False, True, False, False, True]))

In [ ]:
p="/mnt/idms/PROJECTS/Lung/Sinogram-20210901/PA2/ST1/SE5/000000EF"
a=pydicom.read_file(p).pixel_array
plt.imshow(a, cmap="gray")

In [ ]:
a=np.load("/mnt/idms/PROJECTS/Lung/Tudo-Ulyssys-Unzipped/50027/SE000010.npz")
a=a[a.files[0]]
print(a.shape)
for i in range(600):
    img=a[:,:,i]
    print(i)
    plt.imshow(img,cmap=plt.cm.Greys_r)
    plt.show()

In [ ]:
for p in glob.iglob("/mnt/idms/PROJECTS/Lung/Tudo-Ulyssys-Unzipped/*/"):
    p=glob.glob(f"{p}*.npz")[0]
    a=np.load(p)
    a=a[a.files[0]]
    sampled=a[:,:,np.random.randint(a.shape[2],size=10)]
    for i in range(10):
        print(p,i)
        img=sampled[:,:,i]
        plt.imshow(img,cmap=plt.cm.Greys_r)
        plt.show()

In [ ]:
with open("/mnt/idms/PROJECTS/Lung/Tudo-Ulyssys-Unzipped/50014/SE000013.json") as f:
    j = json.load(f)
    print(dict(j))

In [ ]:
a=np.array([[1,2,3],[4,5,6]])
a[:,:1]

In [ ]:
pref="/mnt/idms/PROJECTS/Lung/sinogram_Domi"
for dose in ["020","080","280"]:
    try:
        os.makedirs(f"/mnt/idms/PROJECTS/Lung/Gyula_images/spr_dose{dose}/sinograms")
        os.makedirs(f"/mnt/idms/PROJECTS/Lung/Gyula_images/spr_dose{dose}/slices")
    except:
        pass
    for idx in np.linspace(0,256,10,endpoint=False).astype(int):
        sino=np.load(f"{pref}/spr_dose{dose}/sinograms/{idx}.npy")
        lung=np.load(f"{pref}/spr_dose{dose}/slices/{idx}.npy")
        Image.fromarray(sino).convert('L').save(f"/mnt/idms/PROJECTS/Lung/Gyula_images/spr_dose{dose}/sinograms/{idx}.jpeg", optimize=True)
        Image.fromarray(lung*255).convert('L').save(f"/mnt/idms/PROJECTS/Lung/Gyula_images/spr_dose{dose}/slices/{idx}.jpeg",optimize=True)
        _,axs=plt.subplots(1,2)
        axs[0].imshow(sino,cmap=plt.cm.Greys_r)
        axs[1].imshow(lung,cmap=plt.cm.Greys_r)
        plt.show()

In [ ]:
for p in glob.glob(f"/mnt/idms/PROJECTS/Lung/Tudo-Ulyssys-Unzipped/*/*.npz"):
    arr=np.load(p)
    arr=arr[arr.files[0]]
    slices=np.sort(np.random.choice(arr.shape[2],3,replace=False))
    for i in slices:
        plt.figure()
        plt.imshow(arr[:,:,i], cmap=plt.cm.Greys_r)
        plt.show()
    print(f"CASE {p} FINISHED")

In [ ]:
np.min(arr), np.max(arr)

In [ ]:
orig=(255*(arr-np.min(arr))/(np.max(arr)-np.min(arr))).astype(np.uint8)
smoothed=np.array(Image.fromarray(orig, mode="L").filter(ImageFilter.SMOOTH))#_MORE))
smoothed_more=np.array(Image.fromarray(orig, mode="L").filter(ImageFilter.SMOOTH_MORE))
difference=np.abs(smoothed-orig)
difference_more=np.abs(smoothed_more-orig)
#print(np.unique(difference))
#plt.imshow(difference, cmap=plt.cm.Greys_r)
fig,axs=plt.subplots(1,5, figsize=(25,5))
axs[0].imshow(orig, cmap=plt.cm.Greys_r)
axs[1].imshow(smoothed, cmap=plt.cm.Greys_r)
axs[2].imshow(smoothed_more, cmap=plt.cm.Greys_r)
axs[3].imshow(difference, cmap=plt.cm.Greys_r)
axs[4].imshow(difference_more, cmap=plt.cm.Greys_r)

In [ ]:
Image.fromarray(arr, mode="RGB")

In [ ]:
dose="020"
for slice_number in range(200):
    path=f"/mnt/idms/PROJECTS/Lung/sinogram_Domi/spr_dose{dose}/slices/{slice_number}.npy"
    print(path)
    arr=np.load(path)
    plt.figure(figsize=(25,25))
    plt.imshow(arr,cmap=plt.cm.Greys_r)
    plt.show()

In [ ]:
#Kód Andrásnak: CT szeletek beolvasása
 #"020" or "080" or "280"
slice_number="120" # from 0 up to 255


In [ ]:
import itertools
from multiprocessing import Pool
import functools

def func(a, b):
    print (a,b)

def func_star(a_b):
    """Convert `f([1,2])` to `f(1,2)` call."""
    return func(*a_b)

pool = Pool()
b_args = [1,2,3]
first_arg = 1
f=functools.partial(func,first_arg)
pool.starmap(f,b_args)
#argzip=zip(a_args, itertools.repeat(second_arg))
#pool.starmap(func, argzip)
#pool.starmap(func, [(1,2),(2,3)])

In [ ]:
def f(a,b=5):
    print(a,b)
    
f(2,3)

In [ ]:
#from pydicom.data import get_testdata_files
#filename = get_testdata_files("CT_small.dcm")[0]
ds = pydicom.dcmread(filename)
plt.imshow(ds.pixel_array, cmap=plt.cm.bone) 

In [ ]:
filename = "/mnt/idms/PROJECTS/Lung/Kaggle-data/cancer_cases/home/peter/lung/data/stage1/stage1/229b8b785f880f61d8dad636c3dc2687/f1a23adea3019e89e938f9de24887ae6.dcm"
ds = pydicom.dcmread(filename)
print((ds.pixel_array).shape)

plt.imshow(ds.pixel_array, cmap=plt.cm.bone)

data_img = Image.fromarray(ds.pixel_array)
data_img_rotated = data_img.rotate(angle=45,resample=Image.BICUBIC,fillcolor=data_img.getpixel((0,0)))
print(type(data_img_rotated))
plt.imshow(data_img_rotated)

In [ ]:
from PIL import Image
data_img = Image.fromarray(ds.pixel_array)
data_img_rotated = data_img.rotate(angle=45,resample=Image.BICUBIC,fillcolor=data_img.getpixel((0,0)))

In [ ]:
os.makedirs(os.path.join("./DSB2017/prep_result/","46372/SE3217637").rsplit('/',1)[0], exist_ok=True)

In [ ]:
slices = [pydicom.read_file(s) for s in glob.glob("/mnt/idms/PROJECTS/Lung/Ulyssys-Lung-Data/56473/SE000006/*.dcm")]
slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
for ds in slices:
    plt.figure()
    plt.imshow(ds.pixel_array, cmap=plt.cm.bone)
    plt.show()

In [ ]:
slices = [pydicom.read_file(s) for s in glob.glob("./LymphoglandulaeMetastasis/54841/SE000006/*.dcm")]
slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))


In [ ]:
arr=slices[0].pixel_array
arr_norm = (arr/4095.0)*256
Image.fromarray(arr_norm).show()
print(slices[0].pixel_array.shape)

In [ ]:
np.save("./DSB2017/prep_result/53966/SE000007_clean",np.zeros(4))

In [ ]:
filename_lymp = "/mnt/idms/PROJECTS/Lung/LymphoglandulaeMetastasis/SE000006/50848-0584.dcm"
filename_lung="/mnt/idms/PROJECTS/Lung/LungMetastasis/SE000004/54680-0545.dcm"
lidc="/mnt/idms/PROJECTS/Lung/1-128.dcm"

ds = pydicom.dcmread(lidc)
#print((ds.pixel_array).shape)

plt.imshow(ds.pixel_array, cmap=plt.cm.bone)

#data_img = Image.fromarray(ds.pixel_array)
#data_img_rotated = data_img.rotate(angle=45,resample=Image.BICUBIC,fillcolor=data_img.getpixel((0,0)))
#print(type(data_img_rotated))
#plt.imshow(data_img)

In [ ]:
no_num=len(glob.glob('./NoMetastasis/*.txt'))
lung_num=len(glob.glob('./LungMetastasis/*.txt'))
lymp_num=len(glob.glob('./LymphoglandulaeMetastasis/*.txt'))
ct_num=no_num+lung_num+lymp_num

In [ ]:
lung_labels=np.asarray([0]*no_num+[1]*lung_num+[0]*lymp_num).astype(np.float32)
lymp_labels=np.asarray([0]*no_num+[0]*lung_num+[1]*lymp_num).astype(np.float32)

In [ ]:
glob.glob('./NoMetastasis/*.txt')

In [ ]:


dir_name = './NoMetastasis/'
extension = ".zip"

In [ ]:
os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

In [ ]:
def unzip_nometa(path):
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall(f"{path[:-4]}/")

In [ ]:
import multiprocessing as mp

In [ ]:
pool = mp.Pool(80)
pool.map(unzip_nometa, list(glob.glob('./NoMetastasis/*.zip')))

In [ ]:
for filename in glob.glob('./Tudo-Ulyssys/*.zip'):
    print(filename)
    with tempfile.TemporaryDirectory() as tmpdirname:
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(f"{tmpdirname}/asd")
        print(len([name for name in os.listdir(tmpdirname) if os.path.isfile(name)]))

In [ ]:
s="./Tudo-Ulyssys/55308.zip"

In [ ]:
s[:-3]

In [ ]:
with zipfile.ZipFile(s, 'r') as zip_ref:
    zip_ref.extractall(f"./hello")

In [ ]:
len(glob.glob("/mnt/idms/home/andor/prep_result/*"))

In [ ]:
ff="/mnt/idms/home/andor/bbox_result/*"
for p in glob.glob(ff):
    arr=np.load(p)
    print(arr.shape, np.unique(arr))